In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
#we want overlapping with> lvar, chip, atac. NOT other from gff, lnc

In [ ]:
#still have to check to make sure no ranges overlap within type (eg atac)

In [3]:
#we should also set a minimum length of a range to consider. 50 bp? lower range of enhancers

# important functions

In [4]:
def df2dic(al, intervals):
    ranges = { interval : defaultdict(list) for interval in intervals }
    for Var in intervals:
        temp_df = al[al["type"]==Var]
        for index, row in temp_df.iterrows():
            ranges[Var][row['chr']].append([row['start'],row['stop']])
    return ranges

In [5]:
def unzip(l):
    return list(zip(*l))

In [6]:
def range_merge(A, B):
    A0, A1 = A
    B0, B1 = B
    # return overlap, A', B'
    if( A1 < B0): # no overlap
        return [], [], B
    if(B1 < A0 ): # no overlap
        return [], A, []
    if ( A0 < B0 and B0 <= A1 and A1 <= B1) : # tail left
        return [B0, A1], [], [A1,B1]
    if (B0 <= A0 and A0 <= B1 and B1 < A1): # tail right
        return [A0, B1], [B1, A1], []
    if ( B0 <= A0 and A1 <= B1): # superset A ⊂ B
        return [A0, A1], [], [A1,B1]
    if (A0 < B0 and B1 < A1): # subset B ⊂ A
        return [B0, B1], [B1,A1], []
    else:
        print("ERROR:",A, B)

In [7]:
def interval_to_line(I,y):
    return list(range(I[0],I[1])), [y] * (I[1]-I[0])

def test_intervals(A,B):
    C, A_p, B_p = range_merge(A,B)
    plt.plot(*interval_to_line(A,1), label="A", color="red")
    plt.plot(*interval_to_line(B,0), label="B", color="blue")
    if len(C) > 0:
        plt.plot(*interval_to_line(C,-1), label="C", color="purple")
    if len(A_p) > 0:
        plt.plot(*interval_to_line(A_p,-2), label="A_p", color="red")
    if len(B_p) > 0:
        plt.plot(*interval_to_line(B_p,-3), label="B_p", color="blue")
    plt.legend()
    plt.show()

In [8]:
def compute_overlaps(As,Bs, debug=False):
    ai, bi = 0,0
    overlaps = []
    
    A = As[ai]
    B = Bs[bi]
    safety = 0
    max_safety = 1e6
    while True and safety < max_safety:
        safety += 1
        C, A_p, B_p = range_merge(A,B)

        if len(C) > 0:
            overlaps.append(C)
            if debug:
                print(A,B,C)

        if len(A_p) == 0:
            ai += 1
            if ai >= len(As):
                return overlaps
            else:
                A = As[ai]
        else:
            A = A_p

        if len(B_p) == 0:
            bi += 1
            if bi >= len(Bs):
                return overlaps
            else:
                B = Bs[bi]
        else:
            B = B_p
    
    if safety >= max_safety:
        print("YOU ARE A FAILURE")

In [9]:
def get_merge_results(all_chrs, al, kind1, kind2, ranges):
    ref=ranges[kind1]
    alt=ranges[kind2]
    results = pd.DataFrame() 
    for ch in all_chrs:
        if len(al[(al["chr"]==ch) & (al["type"]==kind1)]) > 0:
            new_contig = compute_overlaps(ref[ch], alt[ch], debug=False)
            for c in new_contig:
                results = results.append({'chr' : ch, 'start' : c[0], 'stop' : c[1]},  
                        ignore_index = True) 
    results.start = results.start.astype(int)
    results.stop = results.stop.astype(int)
    return results

In [10]:
def get_sub_results(all_chrs, al, kind1, kind2, ranges):
    ref=ranges[kind1]
    alt=ranges[kind2]
    results = pd.DataFrame() 
    for ch in all_chrs:
        if len(al[(al["chr"]==ch) & (al["type"]==kind1)]) > 0:
            new_contig = subtract_scaffold(ref[ch], alt[ch])
            for c in new_contig:
                results = results.append({'chr' : ch, 'start' : c[0], 'stop' : c[1]},  
                        ignore_index = True) 
    results.start = results.start.astype(int)
    results.stop = results.stop.astype(int)
    return results

In [11]:
def subtract_scaffold(contig, enemy, debug=False, max_safety=1e6):
    good_idx = 0
    evil_idx = 0
    safety = 0 #this is to avoid an infinite while loop

    while True and safety < max_safety:
        if(good_idx >= len(contig) or evil_idx >= len(enemy)):
            print("YOU ARE A FAILURE.")
            break
        good = contig[good_idx]
        evil = enemy[evil_idx]
        good_start, good_end = good[0], good[1]
        evil_start, evil_end = evil[0], evil[1]
        if debug:
            print(f"checking {good} {evil} :",end=" ")
        if(check_overlap(good,evil)):
            print("OVERLAPPED")
            print(good, evil)
            if debug:
                print("overlap")
            segments = range_takeout(good,evil)
            if debug:
                print("segments",segments)
            contig = contig[:good_idx] + segments + contig[good_idx+1:]
            if debug:
                print("new",contig[good_idx-1:good_idx+2], "current idx", contig[good_idx])
                
            if(good_idx >= len(contig) or evil_idx >= len(enemy)):
                print("extra check saved us.")
                break

        else:
            if(evil_end <= good_start):
                if debug:
                    print("evil")
                evil_idx += 1
                if(evil_idx >= len(enemy)):
                    if debug:
                        print("EVIL FINISHED")
                    break
                if debug:
                    print("next evil", enemy[evil_idx])
            else:
                if debug:
                    print("good")
                good_idx += 1
                if(good_idx >= len(contig)):
                    if debug:
                        print("GOOD FINISHED")
                    break
                if debug:
                    print("next good", contig[good_idx])

        safety += 1
    if debug:
        print(safety)
    return contig

In [12]:
#quick check if two regions overlap
def check_overlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0])) > 0

In [13]:
def range_takeout(contig, evil):
    A0, A1 = contig #region that we like, e.g. conserved region, start and end respectively, start being the lower number no matter the direction of the gene for example
    B0, B1 = evil #region that we don't want in the end, e.g. coding region, start and end
    if( A1 < B0 or B1 < A0 ): # no overlap
        return [contig]
    elif ( A0 < B0 and B0 <= A1 and A1 <= B1) : # tail left
        return [[A0, B0]]
    elif (B0 <= A0 and A0 <= B1 and B1 < A1): # tail right
        return [[B1, A1]]
    elif ( B0 <= A0 and A1 <= B1): # superset
        return []
    elif (A0 < B0 and B1 < A1): # subset
        return [[A0,B0],[B1,A1]]
    else:
        print("ERROR:",contig, evil)

# step 1: overlap between atac and chip

In [14]:
reference = "atac"
toadd = ["chip"]
intervals = [reference] + toadd
intervals

['atac', 'chip']

In [15]:
atac = pd.read_csv("atac_mapped_translatedto5.0.csv", header=None, names=["chr","start","stop"])
chip = pd.read_csv('AlxChip_mapped_translatedto5.0.txt', sep="\t", header=None, names=["chr","start","stop"])

#processing dataframes
atac["type"] = "atac"
chip["type"] = "chip"
atac=atac.sort_values(by=["chr",'start'])
chip=chip.sort_values(by=["chr",'start'])
al = pd.concat([atac, chip])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) <= 0

False

In [16]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [17]:
results=get_merge_results(all_chrs, al, "atac", "chip", ranges)

In [18]:
len(results)

85

In [19]:
results

,chr,start,stop
0,NW_022145514.1,743495,743825
1,NW_022145516.1,1320975,1321168
2,NW_022145520.1,521014,521247
3,NW_022145527.1,183872,184137
4,NW_022145533.1,26015,26218
...,...,...,...
80,NW_022145615.1,10561119,10561413
81,NW_022145615.1,10634069,10634473
82,NW_022145615.1,10654052,10654428
83,NW_022145615.1,10671067,10671460


# step 2: overlap with lvar

In [20]:
reference = "atac_chip"
toadd = ["lvar"]
intervals = [reference] + toadd
intervals

['atac_chip', 'lvar']

In [21]:
lvar = pd.read_csv('lvar_spur_48_50_mapped_translatedto50.csv', header=None, names=["chr","start","stop"])

#processing dataframes
results["type"] = "atac_chip"
lvar["type"] = "lvar"
results=results.sort_values(by=["chr",'start'])
lvar=lvar.sort_values(by=["chr",'start'])
al = pd.concat([results, lvar])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) <= 0

False

In [22]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [23]:
results=get_merge_results(all_chrs, al, "atac_chip", "lvar", ranges)

In [24]:
len(results)

20

In [25]:
results

,chr,start,stop
0,NW_022145514.1,743559,743624
1,NW_022145533.1,26157,26207
2,NW_022145594.1,40376691,40376779
3,NW_022145594.1,40376783,40376838
4,NW_022145594.1,40986392,40986480
5,NW_022145594.1,40986484,40986539
6,NW_022145594.1,41435943,41436041
7,NW_022145594.1,41436041,41436041
8,NW_022145594.1,41436098,41436159
9,NW_022145594.1,41436159,41436159


# step 3: take out lncRNA

In [26]:
reference = "atac_chip"
subtractables = ["lnc"]
intervals = [reference] + subtractables
intervals

['atac_chip', 'lnc']

In [27]:
lnc = pd.read_csv("lncRNA_mapped_translatedto5.0.csv", header=1, names=["chr","start","stop"])

#processing dataframes
results["type"] = "atac_chip"
lnc["type"] = "lnc"
results=results.sort_values(by=["chr",'start'])
lnc=lnc.sort_values(by=["chr",'start'])
al = pd.concat([results, lnc])
all_chrs=al.chr.unique()
al["length"] = al["stop"] - al["start"]

#CHECK IF THERE ARE NEGATIVE NUMBERS
min(al["length"]) <= 0

True

In [28]:
#make it into a dictionary 
ranges = df2dic(al, intervals)

In [29]:
results = get_sub_results(all_chrs, al, "atac_chip", "lnc", ranges)

OVERLAPPED
[7246561, 7246617] [7237563, 7248308]
extra check saved us.


In [30]:
len(results)

19